In [2]:
import sys
import pathlib
import pandas as pd
import numpy as np
import boto3

ROOT = pathlib.Path().absolute().parent.as_posix()
if ROOT not in sys.path:
    sys.path.append(ROOT)

In [3]:
print(sys.path)

['/root/AWS ASDI Competition/ASDI-Hackathon/Scripts', '/opt/conda/lib/python37.zip', '/opt/conda/lib/python3.7', '/opt/conda/lib/python3.7/lib-dynload', '', '/opt/conda/lib/python3.7/site-packages', '/opt/conda/lib/python3.7/site-packages/IPython/extensions', '/root/.ipython', '/root/AWS ASDI Competition/ASDI-Hackathon']


In [ ]:
# !pip install sagemaker-studio-image-build

In [ ]:
# #sm-docker instead of docker after !pip install sagemaker-studio-image-build as known issue running 'docker' in Sagemaker Studio
# import boto3

# account_id = boto3.client('sts').get_caller_identity().get('Account')
# region = boto3.Session().region_name
# ecr_repository = 'asdi-hackathon-sagemaker-processing-container'
# tag = ':latest'
# processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

# # Create ECR repository and push docker image
# !sm-docker build -t $ecr_repository docker
# !aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
# !aws ecr create-repository --repository-name $ecr_repository
# !sm-docker tag {ecr_repository + tag} $processing_repository_uri
# !sm-docker push $processing_repository_uri

In [3]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
ecr_repository = 'asdi-hackathon-sagemaker-processing-container'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

In [1]:
# Create ECR repository and push docker image
#docs wanted you to create docker directory, I have Dockerfile in this directory hence '.'
#remember cant build locally (i.e. VS Code) due to additional Kainos certs required
!docker build -t $ecr_repository .
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

Sending build context to Docker daemon  120.8kB
Step 1/5 : FROM python:3.7-slim-buster
 ---> 15cc9ba5a32a
Step 2/5 : RUN pip3 install pandas scikit-learn numpy multiprocess boto3
 ---> Running in 88cd808f339a
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 KB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.6/500.6 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 KB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 KB 26.

In [ ]:
# Our team avoids putting credentials in repositories, so that means they're not allowed in Dockerfile. Our best practice within applications is to use creds from environment variables.

# We solve for this using docker-compose.

# Within docker-compose.yml, you can specify a file that contains the environment variables for the container:

#  env_file:
# - .env
# Make sure to add .env to .gitignore, then set the credentials within the .env file like:

# SOME_USERNAME=myUser
# SOME_PWD_VAR=myPwd
# Store the .env file locally or in a secure location where the rest of the team can grab it.

#password stored in docker registry?
#used temporary authentication code but still, be aware. Same reason people prefer not to build an image with their credentials locally (which may have to if VPN), rather as part of CI/CD pipeline
#!docker logout

In [1]:
!docker logout

Removing login credentials for https://index.docker.io/v1/


In [4]:
from sagemaker import get_execution_role
from sagemaker.processing import ScriptProcessor

#role = 'role_name_with_sagemaker_permissions' #i.e. my sagemaker role otherwise proceed inside sagemaker and will auto find
role = get_execution_role()

script_processor = ScriptProcessor(command=['python3'], 
                            image_uri=processing_repository_uri, role=role, 
                            instance_type="ml.t3.xlarge", instance_count=1)

In [5]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_processor.run(
    code='processing.py',
    #no inputs technically needed as script is self-sufficient in this regard
    #inputs=[ProcessingInput(source=input_data, destination="asdi-hackathon/*")],
    outputs=[ProcessingOutput(output_name="final_df", source="/opt/ml/processing/output")]
)

processing_job_description = script_processor.jobs[-1].describe()
print(processing_job_description)


Job Name:  asdi-hackathon-sagemaker-processing-con-2022-08-29-11-37-00-480
Inputs:  [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-2-991775080983/asdi-hackathon-sagemaker-processing-con-2022-08-29-11-37-00-480/input/code/processing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'final_df', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-eu-west-2-991775080983/asdi-hackathon-sagemaker-processing-con-2022-08-29-11-37-00-480/output/final_df', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
......................Traceback (most recent call last):
  File "/opt/ml/processing/input/code/processing.py", line 690, in <module>
    create_ai_pickle()
  File "/opt/ml/processing/input/code/processing.py", line 26, in create_ai_pickle
    obj = client.get_object(Bucket=b

UnexpectedStatusException: Error for Processing job asdi-hackathon-sagemaker-processing-con-2022-08-29-11-37-00-480: Failed. Reason: AlgorithmError: See job logs for more information